In [1]:
import os
import pandas as pd
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import undetected_chromedriver as uc
from fake_useragent import UserAgent
import logging
from dotenv import load_dotenv  # Import for loading .env file

In [2]:
# Load environment variables from .env file
load_dotenv()

True

In [3]:
# Setup logging for better traceability
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [4]:
# Constants (move sensitive data like this to environment variables for security in real apps)
MAIL = os.getenv('MAIL')
PASSWORD = os.getenv('PASSWORD')
CHROME_BINARY_LOCATION = "C:\\Users\\benam\\source\\chromedriver-win64\\chromedriver.exe"
CHROMEDRIVER_PATH = "C:\\Users\\benam\\source\\chromedriver-win64\\chromedriver.exe"
CHATGPT_URL = 'https://chatgpt.com/'
COOKIES_PATH = "cookies.pkl"
# PROMPT = "Give a summary of the life and work of Albert Einstein."
# PROMPT =  "Refactor the given Python code to improve its error handling and resilience."
PROMPT =  "Using the following Python code add functionality after logging in to wait for HTTP POST call to be used for input for the PROMPT, the return response for this POST call is the articleOutput[1].text:"

In [5]:
def save_cookies(driver, path):
    """Save cookies to a file."""
    with open(path, 'wb') as filehandler:
        pickle.dump(driver.get_cookies(), filehandler)
    logging.info(f"Cookies saved to {path}")

In [6]:
def load_cookies(driver, path):
    """Load cookies from a file."""
    with open(path, 'rb') as cookiesfile:
        cookies = pickle.load(cookiesfile)
        for cookie in cookies:
            # Selenium expects expiry to be an int
            if isinstance(cookie.get('expiry', None), float):
                cookie['expiry'] = int(cookie['expiry'])
            driver.add_cookie(cookie)
    logging.info(f"Cookies loaded from {path}")

In [7]:
# Set up Chrome options
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument(f"user-agent={UserAgent.random}")
chrome_options.add_argument(r"user-data-dir=C:\\Users\\benam\\AppData\\Local\\Google\\Chrome\\User Data")
# chrome_options.add_experimental_option("detach", True)
# chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
chrome_options.binary_location = CHROME_BINARY_LOCATION

In [8]:
# Start the Chrome driver
logging.info("Starting Chrome browser")
driver = uc.Chrome(chrome_options=chrome_options)
sleep(3)

2024-09-16 00:22:40,055 - INFO - Starting Chrome browser
2024-09-16 00:22:41,028 - INFO - patching driver executable C:\Users\benam\appdata\roaming\undetected_chromedriver\undetected_chromedriver.exe


In [9]:
# Open the ChatGPT website
driver.get(CHATGPT_URL)
sleep(3)

In [10]:
# Attempt to load cookies if they exist
if os.path.exists(COOKIES_PATH):
    logging.info("Loading cookies from file")
    try:
        load_cookies(driver, COOKIES_PATH)
        driver.refresh()  # Refresh to apply cookies
        sleep(5)  # Wait for the page to load after refresh
    except Exception as e:
        logging.error(f"Error loading cookies: {e}")

2024-09-16 00:23:55,054 - INFO - Loading cookies from file
2024-09-16 00:23:55,094 - ERROR - Error loading cookies: Message: unable to set cookie
  (Session info: chrome=128.0.6613.138)
Stacktrace:
	GetHandleVerifier [0x0044D933+25811]
	(No symbol) [0x003DE314]
	(No symbol) [0x002D2399]
	(No symbol) [0x002DCE1B]
	(No symbol) [0x003650C6]
	(No symbol) [0x0033AD2C]
	(No symbol) [0x003546F0]
	(No symbol) [0x0033AAC6]
	(No symbol) [0x0030BEFD]
	(No symbol) [0x0030C8FD]
	GetHandleVerifier [0x0071F143+2981091]
	GetHandleVerifier [0x00772FF9+3324825]
	GetHandleVerifier [0x004DB32F+605903]
	GetHandleVerifier [0x004E2CBC+637020]
	(No symbol) [0x003E6F4D]
	(No symbol) [0x003E3DD8]
	(No symbol) [0x003E3F75]
	(No symbol) [0x003D6406]
	BaseThreadInitThunk [0x75A4FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x76F980CE+286]
	RtlGetAppContainerNamedObjectPath [0x76F9809E+238]



In [11]:
# Click login button
loginBtn = driver.find_element(By.XPATH, "//button[@data-testid='login-button']")
loginBtn.click()
sleep(3)

In [12]:
# Enter email
emailInput = driver.find_element(By.XPATH,"//input[@id='email-input']")
emailInput.send_keys(MAIL)
sleep(3)

In [13]:
# Click continue
continueBtn = driver.find_element(By.XPATH, "//button[@class='continue-btn']")
continueBtn.click()
sleep(3)

In [14]:
# Enter password
password = driver.find_element(By.XPATH, "//input[@id='password']")
password.send_keys(PASSWORD)
sleep(3)

In [15]:
# Submit the login form
submitBtn = driver.find_element(By.XPATH, "//button[@type='submit']")
submitBtn.click()
sleep(10)

In [16]:
# Wait until the prompt input is present, indicating successful login
try:
   wait = WebDriverWait(driver, 120)
   promptInput = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@id='prompt-textarea']")))
   if promptInput:
      logging.info("Login successful")
      save_cookies(driver, COOKIES_PATH)  # Save cookies after successful login
except Exception as e:
   logging.error(f"Error during post-login wait: {e}")
   driver.quit()

2024-09-16 00:25:05,152 - INFO - Login successful
2024-09-16 00:25:05,160 - INFO - Cookies saved to cookies.pkl


In [17]:
# Enter the prompt
promptInput = driver.find_element(By.XPATH, "//div[@id='prompt-textarea']")
promptInput.send_keys(PROMPT)
sleep(3)

In [ ]:
# Send the prompt
promptBtn = driver.find_element(By.XPATH, "//button[@data-testid='send-button']")
promptBtn.click()
sleep(10)

In [23]:
# Capture the output
articleOutput = driver.find_elements(By.CLASS_NAME, "markdown")
print(articleOutput[-1].get_attribute("innerHTML"))
sleep(5)

<p>Albert Einstein (1879–1955) was a German-born theoretical physicist widely regarded as one of the greatest scientific minds of the 20th century. His groundbreaking contributions to the fields of physics revolutionized our understanding of the universe. Here’s a brief overview of his life and work:</p><h3>Early Life and Education:</h3><ul><li><strong>Born</strong> in Ulm, Germany, on March 14, 1879, Einstein grew up in Munich.</li><li>He displayed an early fascination with science, particularly with a compass he was given as a child, which sparked his curiosity about the forces of nature.</li><li>Einstein attended the <strong>Swiss Federal Polytechnic</strong> in Zurich, where he graduated in 1900. Despite struggling to secure a teaching position, he took a job at the <strong>Swiss Patent Office</strong> in Bern in 1902.</li></ul><h3>Major Contributions:</h3><ol><li><p><strong>Annus Mirabilis (1905)</strong> – Einstein’s "Miracle Year":</p><ul><li>In 1905, while working at the patent